# Implement and test image reader

## Imports and configuration

In [1]:
!ln -s ./../lung_cancer_detection

ln: ./lung_cancer_detection: File exists


In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
from monai.config import print_config
from monai.transforms import LoadImage, LoadImaged
from monai.data.image_reader import NumpyReader, ImageReader
from lung_cancer_detection.data.image_reader import LIDCReader

pd.set_option('display.max_columns', None)

DATA_DIR = Path("/Volumes/LaCie/data/lung-cancer-detection/lidc-idri/processed")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
print_config()

MONAI version: 0.4.0
Numpy version: 1.19.4
Pytorch version: 1.7.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 0563a4467fa602feca92d91c7f47261868d171a1

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: 0.18.0
Pillow version: 8.0.1
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.8.2
ITK version: 5.1.2
tqdm version: 4.56.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Specify requirements

In [5]:
?LoadImage

Loading an image requires an `monai.data.image_reader.ImageReader` object at runtime.

In [6]:
??NumpyReader

The default `NumpyReader` does not work for our use case because we want to support the `monai` transformations which relies on metadata from medical images such as orientation.

In [9]:
??ImageReader

In order to create our own `ImageReader`, we have to implement three methods:

- `verify_suffix`: takes a list of filenames and returns a boolean which states whether the file names are supported
- `read`: Read raw image data from specified file(s)
- `get_data`: Extracts data array and meta data from loaded image and returns them

Several particularities of our dataset have to be considered:

- The filename is always a patient ID
- Loading image and mask requires separate calls to the `ImageReader` class
- In addition to reading data from the `npy` files, we need to extract metadata from the dataframe

## Initialize image reader

In [11]:
reader = LIDCReader(DATA_DIR)

In [12]:
filename = "images/unknown.npy"
reader.verify_suffix(filename)

True